In [65]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyreadr
from sklearn.preprocessing import scale
from scipy.stats import t


In [66]:
rw_price = pd.read_csv("rw_price.csv")
rw_returns = pd.read_csv("rw_returns.csv")
log_rw_returns = pd.read_csv("log_rw_returns.csv")

In [67]:
winLen = 200
Nday = len(log_rw_returns)
print(Nday)

S_cov = np.corrcoef(scale(log_rw_returns).T)
S_cov

999


array([[1.        , 0.34982739, 0.32699292, 0.27549521, 0.34886486],
       [0.34982739, 1.        , 0.37785361, 0.33174046, 0.32484628],
       [0.32699292, 0.37785361, 1.        , 0.33200448, 0.34886946],
       [0.27549521, 0.33174046, 0.33200448, 1.        , 0.32947489],
       [0.34886486, 0.32484628, 0.34886946, 0.32947489, 1.        ]])

### Checking Graph operators from 
A Unified Framework for Structured Graph Learning via Spectral Constraints

https://jmlr.org/papers/volume21/19-276/19-276.pdf

In [145]:
def w_from_L(M: np.ndarray) -> np.ndarray:
    M = np.asarray(M)
    if M.ndim != 2 or M.shape[0] != M.shape[1]:
        raise ValueError("M must be a square matrix")

    n = M.shape[0]
    w = np.empty(n * (n - 1) // 2, dtype=M.dtype)
    k = 0
    for i in range(n - 1):            # i = 0 .. n-2
        for j in range(i + 1, n):     # j = i+1 .. n-1 (strict upper)
            w[k] = -M[i, j]
            k += 1
    return w

def L_from_w(w: np.ndarray) -> np.ndarray:
    w = np.asarray(w).ravel()
    k = w.size
    n = int((1 + np.sqrt(1 + 8 * k)) / 2)
    if n * (n - 1) // 2 != k:
        raise ValueError("Invalid length of w: must be n(n-1)/2 for some integer n.")

    Lw = np.zeros((n, n), dtype=float)

    # Fill the strict upper triangle row by row, backwards like in C++
    for i in range(n - 2, -1, -1):
        j = n - i - 1
        # Equivalent of Eigen: Lw.row(i).tail(j) = -w.head(k).tail(j)
        Lw[i, -j:] = -w[:k][-j:]
        k -= j

    # Make symmetric: copy upper triangle into lower
    Lw = Lw + Lw.T

    # Adjust diagonal: subtract column sums
    colsum = Lw.sum(axis=0)
    np.fill_diagonal(Lw, Lw.diagonal() - colsum)

    return Lw

def W_init(M):
    W0 = w_from_L(M)
    W0[W0 < 0] = 0
    return W0

def L_star(Y):

    Y = np.asarray(Y)
    
    if Y.ndim != 2 or Y.shape[0] != Y.shape[1]:
        raise ValueError("Y must be a square matrix.")
    p = Y.shape[0]
    Lstar = np.zeros(int(p*(p-1)/2))

    for i in range(1,p+1):

        for j in range(1,p+1):
           
            if (i > j):
                k = int( (i - j) + ((j - 1)/2)*(2*p - j) ) 
                Lstar[k-1] = Y[i-1,i-1] - Y[i-1, j-1] - Y[ j-1, i-1] + Y[ j-1, j-1]


    return Lstar

def A_from_w(w):

    w = np.asarray(w, dtype=float).ravel()
    k = w.size

    # Infer n from k = n*(n-1)/2
    n_float = (1 + np.sqrt(1 + 8*k)) / 2
    n = int(n_float)
    if n*(n-1)//2 != k:
        raise ValueError(f"len(w)={k} is not a triangular number; expected k=n*(n-1)/2.")

    A = np.zeros((n, n), dtype=float)

    # Indices of strict upper triangle in row-major order
    iu = np.triu_indices(n, k=1)
    A[iu] = w            # fill upper triangle
    A[(iu[1], iu[0])] = w  # mirror to lower triangle
    return A

def w_from_A(M):
    
    N = M.shape[1]
    k = N * (N - 1) // 2
    w = np.zeros(k)
    l = 0

    for i in range(N - 1):
        for j in range(i + 1, N):
            w[l] = M[i, j]
            l += 1
    return w


def A_star(Y):

    Y = np.asarray(Y)
    
    if Y.ndim != 2 or Y.shape[0] != Y.shape[1]:
        raise ValueError("Y must be a square matrix.")
    p = Y.shape[0]
    Astar = np.zeros(int(p*(p-1)/2))

    for i in range(1,p+1):

        for j in range(1,p+1):
           
            if (i > j):
                k = int( (i - j) + ((j - 1)/2)*(2*p - j) ) 
                Astar[k-1] = Y[i-1, j-1] + Y[ j-1, i-1]


    return Astar

def D_star(w: np.ndarray) -> np.ndarray:
    w = np.asarray(w).ravel()
    dStar = L_star(np.diag(w))
    return  dStar

def D_from_w(w: np.ndarray) -> np.ndarray:
    """
    Return the column-wise sums of A(w).
    Equivalent to Eigen: A(w).colwise().sum()
    """
    M = A_from_w(w)               # shape (m, p)
    return M.sum(axis=0)   # shape (p,)

In [69]:
log_rw_returns = pd.read_csv("log_rw_returns.csv")

S_cov = np.corrcoef(scale(log_rw_returns).T)
print("S_cov")
print(S_cov)

S_prec = np.linalg.pinv(S_cov)
print("S_prec")
print(S_prec)

S_cov
[[1.         0.34982739 0.32699292 0.27549521 0.34886486]
 [0.34982739 1.         0.37785361 0.33174046 0.32484628]
 [0.32699292 0.37785361 1.         0.33200448 0.34886946]
 [0.27549521 0.33174046 0.33200448 1.         0.32947489]
 [0.34886486 0.32484628 0.34886946 0.32947489 1.        ]]
S_prec
[[ 1.27263721 -0.24947558 -0.19281752 -0.11937063 -0.25633942]
 [-0.24947558  1.32359962 -0.28620275 -0.21902718 -0.17092178]
 [-0.19281752 -0.28620275  1.32304704 -0.21528053 -0.23040203]
 [-0.11937063 -0.21902718 -0.21528053  1.25090233 -0.22424173]
 [-0.25633942 -0.17092178 -0.23040203 -0.22424173  1.29921337]]


In [146]:
wL = w_from_L(S_prec)
print("w from L")
print(wL)

w = W_init(S_prec)
print("w")
print(w)

Lw = L_from_w(w)
print("Lw from w")
print(Lw) 

Lstar = L_star(Lw)
print("Lstar from Lw")
print(Lstar) 

Aw = A_from_w(w)
print('Aw from w')
print(Aw)

wA = w_from_A(Aw)
print('w from A')
print(wA)

Astar = A_star(Aw)
print('Astar from Aw')
print(Astar)

Dstar = D_star(w)
print('Dstar from w')
print(Dstar)

Dw = D_from_w(w)
print('Dw from w')
print(Dw)

w from L
[0.24947558 0.19281752 0.11937063 0.25633942 0.28620275 0.21902718
 0.17092178 0.21528053 0.23040203 0.22424173]
w
[0.24947558 0.19281752 0.11937063 0.25633942 0.28620275 0.21902718
 0.17092178 0.21528053 0.23040203 0.22424173]
Lw from w
[[ 0.81800315 -0.24947558 -0.19281752 -0.11937063 -0.25633942]
 [-0.24947558  0.9256273  -0.28620275 -0.21902718 -0.17092178]
 [-0.19281752 -0.28620275  0.92470283 -0.21528053 -0.23040203]
 [-0.11937063 -0.21902718 -0.21528053  0.77792007 -0.22424173]
 [-0.25633942 -0.17092178 -0.23040203 -0.22424173  0.88190495]]
Lstar from Lw
[2.24258162 2.12834103 1.83466448 2.21258694 2.42273564 2.14160172
 2.14937582 2.13318397 2.26741184 2.10830848]
Aw from w
[[0.         0.24947558 0.19281752 0.11937063 0.25633942]
 [0.24947558 0.         0.28620275 0.21902718 0.17092178]
 [0.19281752 0.28620275 0.         0.21528053 0.23040203]
 [0.11937063 0.21902718 0.21528053 0.         0.22424173]
 [0.25633942 0.17092178 0.23040203 0.22424173 0.        ]]
w from A


## Code From Paper

In [191]:
log_rw_returns = pd.read_csv("log_rw_returns.csv")

X = scale(log_rw_returns[0:winLen].to_numpy())

# number of nodes
p = X.shape[1] # Predictors
k = 1 # Clusters

# number of observations
T_n = X.shape[0]
sigma_e = np.exp(0.1) # Student T Distribution Standard Dev. 
alpha = 2/(T_n*sigma_e)
beta = 2*np.log(sigma_e)/T_n


S_cov = np.corrcoef(X.T)
S_prec = np.linalg.pinv(S_cov)

w = W_init(S_prec)
A0 = A_from_w(w)
A0 = A0/np.sum(A0,axis=1,keepdims=True)
w = w_from_A(A0)

Lw = L_from_w(w)
Aw = A_from_w(w)

eigVals,eigVecs = np.linalg.eigh(Lw)
U = eigVecs[:,p-k:]

a0 = 1
a = np.repeat(a0, p*(p-1)/2) 
w_lagged = np.repeat(0, p*(p-1)/2)

Theta = Lw.copy()
Phi_n = np.zeros((p,p))

eta = 1e-8

u_n = w - a*w_lagged
mu_vec = np.repeat(0,p*(p-1)/2) # u_n ADMM dual

d = 1 # degree Constraint
z_n = np.repeat(0,p) # Degree Dual 

V = np.zeros((p,k))# p x k  {p = predictors, k = clusters}

rho = 1                 # ADMM hyperparameter. 

mu = 2
tau = 2

# residual vectors
primal_lap_residual = np.array([], dtype=float)
primal_deg_residual = np.array([], dtype=float)
dual_residual       = np.array([], dtype=float)

lagrangian = np.array([], dtype=float)
eta_seq = np.array([], dtype=float)

elapsed_time = np.array([], dtype=float)

In [192]:
def L_update( wn, Phi_n, rho, p, k):

    ''' 
    L_n: Laplace Matrix (Ln = Diag(Wn1) - Wn = Lwn) Laplacian operator “A Unified Framework for Structured Graph Learning via Spectral Constraints”
    Phi_n: ADMM Dual Variable 
    rho: ADMM Hyperparameter
    p: Number of time series
    k: Clusters
    '''
    # Reference: Equation 16
    
    Y = L_from_w(wn) + Phi_n/rho

    eigVals,eigVecs = np.linalg.eigh(Y)
    U = eigVecs[:,p-k:]
    R = np.diag(eigVals[p-k:])
    I = np.eye(R.shape[0]) 
    Ln = 0.5 * U @ ( R + np.sqrt(  R**2 + 4/rho * I  ) ) @ U.T
    return Ln


Ln = L_update( w, Phi_n, rho, p, k)
print(Ln)

[[ 0.78676739  0.16983348 -0.90957676  0.09089385 -0.13791795]
 [ 0.16983348  0.03666066 -0.1963434   0.01962056 -0.0297713 ]
 [-0.90957676 -0.1963434   1.0515559  -0.1050818   0.15944607]
 [ 0.09089385  0.01962056 -0.1050818   0.01050081 -0.01593342]
 [-0.13791795 -0.0297713   0.15944607 -0.01593342  0.0241766 ]]


In [193]:
from scipy.stats import multivariate_t

# mean (mu)
loc = [0., 0., 0., 0., 0.]

# covariance (Sigma)
shape = [
    [1.5,     0.75,    0.375,   0.1875,  0.09375],
    [0.75,    1.5,     0.75,    0.375,   0.1875 ],
    [0.375,   0.75,    1.5,     0.75,    0.375  ],
    [0.1875,  0.375,   0.75,    1.5,     0.75   ],
    [0.09375, 0.1875,  0.375,   0.75,    1.5    ],
]

# degrees of freedom
df = 6.0

# Create a frozen multivariate_t object
# This allows fixing the parameters and then calling methods like rvs()
dist = multivariate_t(loc=loc, shape=shape, df=df)

# Generate samples
num_samples = 1000
samples = dist.rvs(size=num_samples)
samples.shape 

(1000, 5)

In [198]:
import numpy as np
from numpy.linalg import slogdet
from scipy.special import digamma, polygamma, gammaln
# Reference
# https://zouyuxin.github.io/Note/EMtDistribution.pdf
# https://shoichimidorikawa.github.io/Lec/ProbDistr/t-e.pdf
def fit_multivariate_t(
    X,
    nu_init=10.0,
    max_iter=200,
    tol=1e-6,
    fix_nu=None,
    jitter=1e-6,
    verbose=False,
):
    """
    Estimate (mu, Sigma, nu) for a multivariate Student's t via EM.

    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        Data.
    nu_init : float
        Initial degrees of freedom (ignored if fix_nu is not None).
    max_iter : int
        Maximum EM iterations.
    tol : float
        Relative tolerance on log-likelihood for convergence.
    fix_nu : float or None
        If set, keep nu fixed at this value.
    jitter : float
        Diagonal jitter multiplier for numerical stability.
    verbose : bool
        Print progress if True.

    Returns
    -------
    mu : array, shape (p,)
    Sigma : array, shape (p, p)
    nu : float
    history : dict with 'loglik'
    """
    X = np.asarray(X)
    n, p = X.shape

    # Initialize
    mu = X.mean(axis=0)
    # sample covariance (unbiased=False), add tiny jitter
    centered0 = X - mu
    Sigma = centered0.T @ centered0 / n
    Sigma += np.eye(p) * (jitter * np.trace(Sigma) / p + 1e-12)
    nu = float(nu_init if fix_nu is None else fix_nu)

    def mahal_sq(Sigma, X, mu):
        # δ_i = (x_i - mu)^T Sigma^{-1} (x_i - mu)
        L = np.linalg.cholesky(Sigma)
        Y = np.linalg.solve(L, (X - mu).T)   # shape (p, n)
        return np.sum(Y * Y, axis=0)         # shape (n,)

    def loglik(mu, Sigma, nu):
        # log-likelihood of multivariate t
        sgn, logdet = slogdet(Sigma)
        if sgn <= 0:
            return -np.inf
        delta = mahal_sq(Sigma, X, mu)
        term1 = gammaln((nu + p) / 2) - gammaln(nu / 2)
        term2 = - (p / 2) * np.log(nu * np.pi) - 0.5 * logdet
        term3 = - ((nu + p) / 2) * np.log1p(delta / nu)
        return np.sum(term1 + term2 + term3)

    ll_old = -np.inf
    history = {'loglik': []}

    for it in range(1, max_iter + 1):
        # E-step
        delta = mahal_sq(Sigma, X, mu)
        w = (nu + p) / (nu + delta)  # E[lambda_i | x_i]
        eloglam = digamma((nu + p) / 2) - np.log((nu + delta) / 2)  # E[log lambda_i | x_i]

        # M-step: mu (weighted mean)
        sumw = w.sum()
        mu = (w[:, None] * X).sum(axis=0) / sumw

        # M-step: Sigma
        Xc = X - mu
        Sigma = (Xc.T * w) @ Xc / n
        # stabilize
        Sigma += np.eye(p) * (jitter * np.trace(Sigma) / p + 1e-12)

        # M-step: nu (solve for root of dQ/dnu = 0) unless fixed
        if fix_nu is None:
            # f(nu) = log(nu/2) - psi(nu/2) + 1 + (1/n) * sum(E[log lambda_i] - E[lambda_i]) = 0
            c = (eloglam.mean() - w.mean())

            def f(nu_):
                return np.log(nu_ / 2.0) - digamma(nu_ / 2.0) + 1.0 + c

            def fprime(nu_):
                return 1.0 / nu_ - 0.5 * polygamma(1, nu_ / 2.0)

            # Newton update with simple guarding
            nu_new = max(nu, 2.01)  # keep > 2 so covariance exists
            for _ in range(30):
                val = f(nu_new)
                der = fprime(nu_new)
                step = val / der
                nu_new = nu_new - step
                if nu_new < 2.01:
                    nu_new = 2.01
                if abs(step) / nu_new < 1e-6:
                    break
            nu = float(nu_new)
        else:
            nu = float(fix_nu)

        # Evaluate log-likelihood and check convergence
        ll = loglik(mu, Sigma, nu)
        history['loglik'].append(ll)
        if verbose:
            print(f"iter {it:3d}: ll={ll:.6f}, nu={nu:.4f}")

        if np.isfinite(ll_old):
            if abs(ll - ll_old) / (abs(ll_old) + 1e-12) < tol:
                break
        ll_old = ll

    return mu, Sigma, nu, history







# X: (n_samples, n_features) data

mu, Sigma, nu, history = fit_multivariate_t(
samples,
nu_init=30.0,
max_iter=200,
tol=1e-6,
fix_nu=None,
jitter=1e-6,
verbose=False,
)
print("mu:", mu)
print("nu:", nu)
print("Sigma:\n", Sigma)

mu: [-0.02766317 -0.07871325  0.0247867   0.02145099 -0.00145958]
nu: 6.377364535231853
Sigma:
 [[1.43181152 0.62514321 0.31916069 0.13240914 0.09206454]
 [0.62514321 1.38117318 0.69953388 0.35969955 0.23922112]
 [0.31916069 0.69953388 1.46624374 0.73770786 0.46042783]
 [0.13240914 0.35969955 0.73770786 1.49563501 0.74303463]
 [0.09206454 0.23922112 0.46042783 0.74303463 1.42116393]]


In [219]:
import numpy as np
from scipy.special import digamma, polygamma, gammaln

def _update_nu_univariate(E_eta, E_log_eta, nu0, max_iter=50, tol=1e-8):
    target = 1.0 + np.mean(E_log_eta) - np.mean(E_eta)
    nu = max(float(nu0), 1e-6)
    for _ in range(max_iter):
        f  = digamma(nu/2.0) - np.log(nu/2.0) - target
        df = 0.5 * polygamma(1, nu/2.0) - 1.0/nu
        step = f / df
        nu_new = max(nu - step, 1e-8)
        if abs(nu_new - nu) < tol * (1.0 + abs(nu)):
            return nu_new
        nu = nu_new
    return nu

def em_student_t_univariate(x, mu=None, lam=None, nu=10.0,
                            max_iter=200, tol=1e-6, verbose=False):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]

    if mu  is None: mu  = x.mean()
    if lam is None: lam = 1.0 / np.var(x)   # precision of t's scale s^2 (not variance)
    nu = float(nu)

    loglik_trace = []
    prev_ll = -np.inf

    for it in range(1, max_iter+1):
        # Save old params for param-change stop
        mu_prev, lam_prev, nu_prev = mu, lam, nu

        # -------- E-step --------
        q = lam * (x - mu)**2
        E_eta     = (nu + 1.0) / (nu + q)
        E_log_eta = digamma((nu + 1.0)/2.0) - np.log((nu + q)/2.0)

        # -------- M-step --------
        mu  = np.sum(E_eta * x) / np.sum(E_eta)
        lam = N / np.sum(E_eta * (x - mu)**2)
        nu  = _update_nu_univariate(E_eta, E_log_eta, nu)

        # -------- Recompute q with UPDATED params --------
        q = lam * (x - mu)**2

        # -------- Log-likelihood with UPDATED params --------
        ll = (
            gammaln((nu + 1.0)/2.0) - gammaln(nu/2.0)
            + 0.5*np.log(lam / (np.pi * nu))
            - ((nu + 1.0)/2.0) * np.log1p(q / nu)
        ).sum()
        loglik_trace.append(ll)

        if verbose:
            print(f"[{it:03d}] ll={ll:.6f}, mu={mu:.3f}, lam={lam:.3f}, nu={nu:.3f}")

        # Convergence: ΔLL and parameter change
        param_change = max(abs(mu - mu_prev), abs(lam - lam_prev), abs(nu - nu_prev))
        if (np.isfinite(prev_ll) and ll - prev_ll < tol * (1.0 + abs(prev_ll))) or (param_change < 1e-7):
            break
        prev_ll = ll

    return mu, lam, nu, np.array(loglik_trace)


import numpy as np
from scipy.stats import t

# --- Generate synthetic univariate Student-t data ---
rng = np.random.default_rng(0)

N = 5000
true_mu = 0.0     # location
true_sigma = 2.0  # scale (std dev-like, not precision)
true_nu = 5.0     # degrees of freedom

# scipy's t takes df=nu, loc=mu, scale=sigma
x = t.rvs(df=true_nu, loc=true_mu, scale=true_sigma, size=N, random_state=rng)


# Fit with EM
mu_hat, lam_hat, nu_hat, ll = em_student_t_univariate(x, mu=None, lam=0.5, nu=8,
                            max_iter=200, tol=1e-6, verbose=True)
print("\nEstimated parameters:")
print("mu =", mu_hat)
print("sigma^2 ≈", np.sqrt(1/lam_hat))
print("nu =", nu_hat)


[001] ll=-11529.219245, mu=0.005, lam=0.299, nu=6.408
[002] ll=-11491.897471, mu=0.003, lam=0.261, nu=6.053
[003] ll=-11488.705740, mu=0.003, lam=0.252, nu=5.914
[004] ll=-11488.126385, mu=0.003, lam=0.250, nu=5.830
[005] ll=-11487.790087, mu=0.003, lam=0.250, nu=5.765
[006] ll=-11487.506431, mu=0.003, lam=0.250, nu=5.707
[007] ll=-11487.256721, mu=0.003, lam=0.251, nu=5.655
[008] ll=-11487.036339, mu=0.003, lam=0.252, nu=5.606
[009] ll=-11486.842075, mu=0.003, lam=0.253, nu=5.561
[010] ll=-11486.671086, mu=0.003, lam=0.253, nu=5.519
[011] ll=-11486.520800, mu=0.003, lam=0.254, nu=5.480
[012] ll=-11486.388888, mu=0.003, lam=0.254, nu=5.444
[013] ll=-11486.273251, mu=0.003, lam=0.255, nu=5.411
[014] ll=-11486.172002, mu=0.003, lam=0.255, nu=5.379
[015] ll=-11486.083449, mu=0.003, lam=0.256, nu=5.350
[016] ll=-11486.006083, mu=0.003, lam=0.256, nu=5.323
[017] ll=-11485.938557, mu=0.002, lam=0.257, nu=5.298
[018] ll=-11485.879673, mu=0.002, lam=0.257, nu=5.275
[019] ll=-11485.828371, mu=0

In [244]:
import numpy as np
from numpy.linalg import inv, pinv, slogdet, LinAlgError
from scipy.special import digamma, polygamma, gammaln

# ---------- helpers ----------
def _quadforms(X, mu, Lambda):
    """q_i = (x_i - mu)^T Lambda (x_i - mu) for all rows."""
    diff = X - mu[None, :]
    y = diff @ Lambda  # (N,D)
    return np.einsum("ni,ni->n", diff, y)

def _loglik_student_t(X, mu, Lambda, nu):
    """Total log-likelihood under multivariate Student-t (precision form)."""
    N, D = X.shape
    q = _quadforms(X, mu, Lambda)
    sign, logdet_L = slogdet(Lambda)
    if sign <= 0:
        return -np.inf
    c = (
        gammaln((nu + D) / 2.0)
        - gammaln(nu / 2.0)
        + 0.5 * logdet_L
        - (D / 2.0) * np.log(nu * np.pi)
    )
    return np.sum(c - 0.5 * (nu + D) * np.log1p(q / nu))

def _update_nu(E_eta, E_log_eta, nu0, max_iter=50, tol=1e-8):
    """
    Newton–Raphson for: psi(nu/2) - log(nu/2) = 1 + mean(E[log eta]) - mean(E[eta]).
    """
    target = 1.0 + float(np.mean(E_log_eta)) - float(np.mean(E_eta))
    nu = max(float(nu0), 1e-6)
    for _ in range(max_iter):
        f  = digamma(nu / 2.0) - np.log(nu / 2.0) - target
        df = 0.5 * polygamma(1, nu / 2.0) - 1.0 / nu
        step = f / df
        nu_new = max(nu - step, 1e-8)
        if abs(nu_new - nu) < tol * (1.0 + abs(nu)):
            return nu_new
        nu = nu_new
    return nu

def _safe_inv(S):
    """Try inv, fall back to pinv if S is near-singular."""
    try:
        return inv(S)
    except LinAlgError:
        return pinv(S)

# ---------- EM main ----------
def em_multivariate_student_t(
    X, mu=None, Lambda=None, nu=10.0,
    max_iter=200, tol=1e-6, jitter=1e-6, verbose=False
):
    """
    Fit a multivariate Student's t via EM (precision parameterization).
    
    Parameters
    ----------
    X : (N,D) array
        Data.
    mu : (D,) or None
        Initial mean (defaults to sample mean).
    Lambda : (D,D) or None
        Initial precision (defaults to inverse sample covariance).
    nu : float
        Initial degrees of freedom (>0).
    max_iter : int
        Max EM iterations.
    tol : float
        Relative LL tolerance for convergence.
    jitter : float
        Base jitter added to the scatter before inversion (stability).
    verbose : bool
        Print progress if True.
        
    Returns
    -------
    mu, Lambda, nu, loglik_trace
    """
    X = np.asarray(X, dtype=float)
    N, D = X.shape

    # --- init ---
    if mu is None:
        mu = X.mean(axis=0)
    if Lambda is None:
        S0 = np.cov(X.T, bias=False)
        S0 = 0.5 * (S0 + S0.T)
        S0 += jitter * np.trace(S0) / D * np.eye(D)
        Lambda = _safe_inv(S0)
    nu = float(nu)

    loglik_trace = []
    prev_ll = -np.inf

    for it in range(1, max_iter + 1):
        mu_old, Lambda_old, nu_old = mu.copy(), Lambda.copy(), nu

        # ----- E-step -----
        q = _quadforms(X, mu, Lambda)              # (N,)
        E_eta = (nu + D) / (nu + q)                # weights E[eta_i]
        E_log_eta = digamma((nu + D) / 2.0) - np.log((nu + q) / 2.0)

        # ----- M-step -----
        # mean (weighted)
        wsum = E_eta.sum()
        mu = (X * E_eta[:, None]).sum(axis=0) / wsum

        # weighted scatter -> precision
        diff = X - mu[None, :]
        S = (diff * E_eta[:, None]).T @ diff / N
        # symmetrize and stabilize
        S = 0.5 * (S + S.T)
        S += jitter * np.trace(S) / D * np.eye(D)
        Lambda = _safe_inv(S)
        # symmetrize Lambda too (tiny numerical asymmetries)
        Lambda = 0.5 * (Lambda + Lambda.T)

        # update nu (scalar)
        nu = _update_nu(E_eta, E_log_eta, nu)

        # ----- log-likelihood with updated params -----
        ll = _loglik_student_t(X, mu, Lambda, nu)
        loglik_trace.append(ll)
        if verbose:
            print(f"[{it:03d}] ll={ll:.6f}  nu={nu:.4f}")

        # convergence: ΔLL and parameter movement
        param_delta = max(
            np.linalg.norm(mu - mu_old, ord=np.inf),
            np.linalg.norm(Lambda - Lambda_old, ord='fro') / (1e-12 + np.linalg.norm(Lambda_old, ord='fro')),
            abs(nu - nu_old) / (1e-12 + abs(nu_old))
        )
        if (np.isfinite(prev_ll) and ll - prev_ll < tol * (1.0 + abs(prev_ll))) or (param_delta < 1e-7):
            break
        prev_ll = ll

    return mu, Lambda, nu, np.array(loglik_trace)


# -------------------- demo / usage --------------------
if __name__ == "__main__":
    from scipy.stats import multivariate_t

    # mean (mu)
    loc = np.zeros(5)

    # "shape" matrix (Σ) used by scipy's multivariate_t (this is NOT the covariance)
    shape = np.array([
        [1.5,     0.75,    0.375,   0.1875,  0.09375],
        [0.75,    1.5,     0.75,    0.375,   0.1875 ],
        [0.375,   0.75,    1.5,     0.75,    0.375  ],
        [0.1875,  0.375,   0.75,    1.5,     0.75   ],
        [0.09375, 0.1875,  0.375,   0.75,    1.5    ],
    ])
    Lambda_true = inv(shape)   # precision in the EM model
    df = 6.0

    print("\nTrue mu:\n", loc)
    print("\nTrue precision Lambda:\n", Lambda_true)
    print("\nTrue nu:\n", df)

    # Generate samples
    rng = np.random.default_rng(0)
    dist = multivariate_t(loc=loc, shape=shape, df=df, seed=rng)
    num_samples = 200000  # more data → better ν
    samples = dist.rvs(size=num_samples)

    # Fit
    mu_hat, Lambda_hat, nu_hat, ll = em_multivariate_student_t(
        samples, mu=None, Lambda=None, nu=8.0,
        max_iter=300, tol=1e-10, jitter=1e-6, verbose=False
    )

    print("\nEstimated mu:\n", mu_hat)
    print("\nEstimated precision Lambda:\n", Lambda_hat)
    print("\nEstimated nu:\n", nu_hat)

    # simple diagnostics
    rel_err = np.linalg.norm(Lambda_hat - Lambda_true, 'fro') / np.linalg.norm(Lambda_true, 'fro')
    print("\nRelative Frobenius error vs true Λ:", rel_err)
    print("Max |mu_hat - mu_true|:", np.max(np.abs(mu_hat - loc)))



True mu:
 [0. 0. 0. 0. 0.]

True precision Lambda:
 [[ 8.88888889e-01 -4.44444444e-01 -3.70074342e-17 -1.85037171e-17
   0.00000000e+00]
 [-4.44444444e-01  1.11111111e+00 -4.44444444e-01  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00 -4.44444444e-01  1.11111111e+00 -4.44444444e-01
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.44444444e-01  1.11111111e+00
  -4.44444444e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.44444444e-01
   8.88888889e-01]]

True nu:
 6.0

Estimated mu:
 [ 0.00335245  0.00319906  0.00281707  0.0007678  -0.00179385]

Estimated precision Lambda:
 [[ 8.91504286e-01 -4.49066319e-01  3.61762082e-03  5.43643906e-04
   7.20005726e-04]
 [-4.49066319e-01  1.11777088e+00 -4.49207893e-01  4.62460191e-03
  -3.25564965e-03]
 [ 3.61762082e-03 -4.49207893e-01  1.11911552e+00 -4.51158122e-01
   2.34583100e-03]
 [ 5.43643906e-04  4.62460191e-03 -4.51158122e-01  1.11597787e+00
  -4.46948765e-01]
 [ 7.20005726e-04 -3.25564965e-03  2.34583100e-03 -4.46948

### w update 

In [261]:
S_sum = 0
x = samples[0:winLen,:]
Lw = L_from_w(w)

# can be replaced by an einsum
for t in range(0,winLen):
    S_sum += ( np.outer(x[t,:],x[t,:]) )/( x[t,:].T @ Lw @ x[t,:] + nu )
C = (p + nu)/winLen
S_tilde = C * S_sum


aw = L_star(S_tilde + Phi_n + rho*(Lw - Ln) + eta*np.outer(V,V.T) )
bw = -u_n - rho*( u_n + np.multiply(a,w_lagged) ) + D_star(z_n - rho*(d - D_from_w(w) ))
cw = (1 - rho/(rho*(4*rho -1) ) ) * w - 1/( rho*(4*rho - 1) ) * (aw + bw)
cth = np.sqrt((2*beta)/(rho*(4*rho - 1))) * np.repeat(1,p*(p-1)/2)

Wupdate = np.multiply((cw > cth),cw)
Wupdate

array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])

In [263]:

def softThresh(v,thr):
    temp = abs(v) - thr
    temp = temp*(temp>0)
    return(np.sign(v)*temp)



In [ ]:
def w_update( x, beta, rho, a, w_n_prev, w_n, L_n, Vn, p, k, d, Tn, nu, eta, Phi_n, mu_n, z_n):

    ''' 
    x: observation for each time series
    p: Number of Time Series
    a: VAR coefficients
    k: Number of Clusters
    nu: Student T degrees of freedom
    Vn: Laplacian spectral constraint
    Tn: Number of observations in a Frame 
    eta: regularization term
    d: Node degree constraint

    Phi_n: ADMM dual variable
    mu_n: ADMM dual variable
    z_n: ADMM dual variable
    '''

    assert Vn.shape[0] == p and Vn.shape[1] == k

    Lstar = L_star(w) 
    dstar = D_star(w)

    u_n = w_n - a * w_n_prev # Vector Autoregressive variable

    cth = np.sqrt( (2*beta)/(rho*(4*rho -1)) ) * np.ones(len(w_n))

    S_sum = 0
    for i in range(0,Tn):
        S_sum += ( x[i,:]*x[i,:].T )/( x[i,:].T @ L_n @ x[i,:] + nu )

    S = ( (p + nu)/Tn ) * S_sum

    aw = L_star @ ( S + Phi_n + rho*(L_star - L_n) + eta * Vn@Vn.T )

    bw = -mu_n - rho*( u_n + a*w_n_prev ) + d_star @ ( z_n - rho * ( d - d_star*w_n ) )

    cw = (1 - (rho)/(rho*(4*p-1))) * wn - (1)/( rho*(4*p -1) ) * (aw + bw)

    wn = (cw > cth) * cw

    return wn